# Change Point Index Creator and Day Splitter

In [1]:
import pandas as pd
import os
import sys
sys.path.append("..")

os.chdir("..")
# List all CSV files in the data/raw directory
csv_files = [f for f in os.listdir('data/raw') if f.endswith('.csv')]

# ,date,sender_id,bgl,bgl_date_millis,text,template,msg_type,affects_fob,affects_iob,dose_units,food_g,food_glycemic_index,dose_automatic,fp_bgl,message_basal_change,__typename,trend
# Define the columns to load
columns_to_load = ['date', 'bgl', 'msg_type', 'affects_fob', 'affects_iob','dose_units','food_g','food_glycemic_index']

# Load each CSV file into a DataFrame and store them in a dictionary
dataframes = {file: pd.read_csv(os.path.join('data/raw', file), usecols=columns_to_load, parse_dates=['date']) for file in csv_files}

# Print the names of the loaded DataFrames
print("Loaded DataFrames:", list(dataframes.keys()))

Loaded DataFrames: ['500030_2024-07-01_2024-09-30.csv', '679372_2024-07-01_2024-09-30.csv']


In [2]:
cr_df = dataframes['679372_2024-07-01_2024-09-30.csv']
cr_df.head()

,date,bgl,msg_type,affects_fob,affects_iob,dose_units,food_g,food_glycemic_index
0,2024-07-01 00:02:32-04:00,115.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-07-01 00:05:33-04:00,112.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-07-01 00:08:33-04:00,116.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-07-01 00:10:34-04:00,121.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-07-01 00:13:36-04:00,122.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
cr_df[cr_df['msg_type'] == 'ANNOUNCE_MEAL']

,date,bgl,msg_type,affects_fob,affects_iob,dose_units,food_g,food_glycemic_index
256,2024-07-01 13:39:01.062000-04:00,135.0,ANNOUNCE_MEAL,True,False,0.0,22.0,0.5
275,2024-07-01 14:32:31.740000-04:00,157.0,ANNOUNCE_MEAL,True,False,0.0,22.0,0.5
308,2024-07-01 15:44:35.026000-04:00,223.0,ANNOUNCE_MEAL,True,False,0.0,22.0,0.5
718,2024-07-02 13:19:08.176000-04:00,102.0,ANNOUNCE_MEAL,True,False,0.0,40.0,0.5
849,2024-07-02 20:12:00.212000-04:00,97.0,ANNOUNCE_MEAL,True,False,NaN,15.0,0.5
...,...,...,...,...,...,...,...,...
39619,2024-09-29 20:44:47.993000-04:00,69.0,ANNOUNCE_MEAL,True,False,NaN,12.0,0.5
39620,2024-09-29 20:46:03.006000-04:00,69.0,ANNOUNCE_MEAL,True,False,NaN,6.0,0.5
39641,2024-09-29 21:32:43.092000-04:00,89.0,ANNOUNCE_MEAL,True,False,NaN,6.0,0.5
39920,2024-09-30 14:18:29.569000-04:00,132.0,ANNOUNCE_MEAL,True,False,NaN,10.0,0.5


In [4]:
cr_df = cr_df.set_index('date').squeeze()
cr_df.index = pd.DatetimeIndex(cr_df.index)

In [5]:
meal_annoucements_df = cr_df[cr_df['msg_type'] == 'ANNOUNCE_MEAL']
meal_annoucements_df


,bgl,msg_type,affects_fob,affects_iob,dose_units,food_g,food_glycemic_index
date,,,,,,,
2024-07-01 13:39:01.062000-04:00,135.0,ANNOUNCE_MEAL,True,False,0.0,22.0,0.5
2024-07-01 14:32:31.740000-04:00,157.0,ANNOUNCE_MEAL,True,False,0.0,22.0,0.5
2024-07-01 15:44:35.026000-04:00,223.0,ANNOUNCE_MEAL,True,False,0.0,22.0,0.5
2024-07-02 13:19:08.176000-04:00,102.0,ANNOUNCE_MEAL,True,False,0.0,40.0,0.5
2024-07-02 20:12:00.212000-04:00,97.0,ANNOUNCE_MEAL,True,False,NaN,15.0,0.5
...,...,...,...,...,...,...,...
2024-09-29 20:44:47.993000-04:00,69.0,ANNOUNCE_MEAL,True,False,NaN,12.0,0.5
2024-09-29 20:46:03.006000-04:00,69.0,ANNOUNCE_MEAL,True,False,NaN,6.0,0.5
2024-09-29 21:32:43.092000-04:00,89.0,ANNOUNCE_MEAL,True,False,NaN,6.0,0.5


In [6]:
meal_annoucements_df = meal_annoucements_df.resample('5min').first()
meal_annoucements_df

,bgl,msg_type,affects_fob,affects_iob,dose_units,food_g,food_glycemic_index
date,,,,,,,
2024-07-01 13:35:00-04:00,135.0,ANNOUNCE_MEAL,True,False,0.0,22.0,0.5
2024-07-01 13:40:00-04:00,NaN,None,None,None,NaN,NaN,NaN
2024-07-01 13:45:00-04:00,NaN,None,None,None,NaN,NaN,NaN
2024-07-01 13:50:00-04:00,NaN,None,None,None,NaN,NaN,NaN
2024-07-01 13:55:00-04:00,NaN,None,None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2024-09-30 17:45:00-04:00,NaN,None,None,None,NaN,NaN,NaN
2024-09-30 17:50:00-04:00,NaN,None,None,None,NaN,NaN,NaN
2024-09-30 17:55:00-04:00,NaN,None,None,None,NaN,NaN,NaN


In [7]:
meal_annoucements_df = meal_annoucements_df[meal_annoucements_df['msg_type'] == 'ANNOUNCE_MEAL']
meal_annoucements_df

,bgl,msg_type,affects_fob,affects_iob,dose_units,food_g,food_glycemic_index
date,,,,,,,
2024-07-01 13:35:00-04:00,135.0,ANNOUNCE_MEAL,True,False,0.0,22.0,0.5
2024-07-01 14:30:00-04:00,157.0,ANNOUNCE_MEAL,True,False,0.0,22.0,0.5
2024-07-01 15:40:00-04:00,223.0,ANNOUNCE_MEAL,True,False,0.0,22.0,0.5
2024-07-02 13:15:00-04:00,102.0,ANNOUNCE_MEAL,True,False,0.0,40.0,0.5
2024-07-02 20:10:00-04:00,97.0,ANNOUNCE_MEAL,True,False,NaN,15.0,0.5
...,...,...,...,...,...,...,...
2024-09-29 20:40:00-04:00,71.0,ANNOUNCE_MEAL,True,False,NaN,6.0,0.5
2024-09-29 20:45:00-04:00,69.0,ANNOUNCE_MEAL,True,False,NaN,6.0,0.5
2024-09-29 21:30:00-04:00,89.0,ANNOUNCE_MEAL,True,False,NaN,6.0,0.5


In [8]:
non_meal_df = cr_df[cr_df['msg_type'] != 'ANNOUNCE_MEAL']
non_meal_df = non_meal_df.resample('5min').first()

cr_resampled = non_meal_df.join(meal_annoucements_df, how='left', rsuffix='_meal')

print(cr_resampled[cr_resampled['msg_type_meal'] == 'ANNOUNCE_MEAL'])


                             bgl      msg_type affects_fob affects_iob  \
date                                                                     
2024-07-01 13:35:00-04:00  135.0  DOSE_INSULIN       False        True   
2024-07-01 14:30:00-04:00  157.0          None        None        None   
2024-07-01 15:40:00-04:00  223.0          None        None        None   
2024-07-02 13:15:00-04:00  102.0          None        None        None   
2024-07-02 20:10:00-04:00   97.0  DOSE_INSULIN       False        True   
...                          ...           ...         ...         ...   
2024-09-29 20:40:00-04:00   71.0          None        None        None   
2024-09-29 20:45:00-04:00   68.0          None        None        None   
2024-09-29 21:30:00-04:00   89.0          None        None        None   
2024-09-30 14:15:00-04:00  132.0  DOSE_INSULIN       False        True   
2024-09-30 18:05:00-04:00   92.0          None        None        None   

                           dose_units

In [9]:
cr_resampled.loc['2024-07-01 14:30:00-04:00']

bgl                                 157.0
msg_type                             None
affects_fob                          None
affects_iob                          None
dose_units                            NaN
food_g                                NaN
food_glycemic_index                   NaN
bgl_meal                            157.0
msg_type_meal               ANNOUNCE_MEAL
affects_fob_meal                     True
affects_iob_meal                    False
dose_units_meal                       0.0
food_g_meal                          22.0
food_glycemic_index_meal              0.5
Name: 2024-07-01 14:30:00-04:00, dtype: object